# **1. Introduction, initial data analysis and visualization about 2025**

Since 2011 Stackoverflow is doing annual Developer Survey [source](https://survey.stackoverflow.co/). The 2025 Developer Survey is the definitive report on the state of software development. In its fifteenth year, Stack Overflow received more than **49 thousand** responses from 177 countries across 62 questions focused on 314 different technologies, including new focus on AI agent tools, LLMs and community platforms [source](https://survey.stackoverflow.co/2025). Among respondents, approximately **2,900** were **data professionals**: Data scientist, AI/ML engineer, Data engineer, Data or business analyst, Financial analyst or engineer, Database administrator or engineer, and Applied scientist.

---
## **Import of packages and data**

In [ ]:
import pandas as pd
import glob, os, re
import plotly.express as px

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
# Load data
df = pd.read_csv(
    r"Stack-overflow-datasets\2025 Stack Overflow Survey Results.csv",
    low_memory=False
)

print(df.shape)
df.head()

---
## **Initial data analysis**

In [ ]:
df.info()

In [ ]:
df["Country"].nunique()

In [ ]:
# Display columns with index numbers, for easier way to keep neccessary columns
for idx, col in enumerate(df.columns):
    print(f"{idx}: {col}")

In [ ]:
# Keeping columns by index
df = df.iloc[:, [2, 3, 6, 8, 11, 12, 15, 40, 57, 61, 65, 66, 71, 77, 170]]

df.head(1)

In [ ]:
df.DevType.unique()

In [ ]:
# These roles are chosen to broadly cover data professionals
roles = [
    "Data scientist",
    "AI/ML engineer",
    "Data engineer",
    "Data or business analyst",
    "Financial analyst or engineer",
    "Database administrator or engineer",
    "Applied scientist"
]

df_ds = df[df["DevType"].isin(roles)]

In [ ]:
df_ds.shape

In [ ]:
# Count data professionals by country
country_counts = df_ds['Country'].value_counts().head(10)

# Create dataframe for plotly
country_df = pd.DataFrame({
    'Country': country_counts.index,
    'Count': country_counts.values
})

fig_countries = px.pie(
    country_df,
    names='Country',
    values='Count',
    title='Geographic Distribution of Data Professionals in 2025<br><sub>Top 10 Countries in Stack Overflow 2025 Survey</sub>',
    hole=0.3
)

fig_countries.update_traces(
    textposition='inside',
    textinfo='percent',
    textfont=dict(size=18, color='white'),
    marker=dict(line=dict(color='white', width=3)),
    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
)

fig_countries.update_layout(
    template='plotly_white',
    height=550,
    showlegend=True,
    font=dict(size=16),
    title_font=dict(size=26),
    legend=dict(font=dict(size=15)),
    margin=dict(l=20, r=20, t=110, b=40)
)

fig_countries.show()

In [ ]:
# Count how many people are in each role
role_counts = df_ds["DevType"].value_counts()

# Create dataframe for plotly
role_df = pd.DataFrame({
    'Role': role_counts.index,
    'Count': role_counts.values
})

fig_roles = px.pie(
    role_df,
    names='Role',
    values='Count',
    title='Distribution of Data Professional Roles in 2025<br><sub>Stack Overflow Survey</sub>',
    hole=0.3)

fig_roles.update_traces(
    textposition='inside',
    textinfo='percent',
    textfont=dict(size=13, color='white'),
    marker=dict(line=dict(color='white', width=3)),
    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
)

fig_roles.update_layout(
    template='plotly_white',
    height=550,
    showlegend=True,
    font=dict(size=16),
    title_font=dict(size=26),
    legend=dict(font=dict(size=15)),
    margin=dict(l=20, r=20, t=110, b=40)
)

fig_roles.show()

---
## **Visualization**

---
### Distribution of Data Professional Roles and AI threat

In [ ]:
# AI Threat Perception
ai_threat = df_ds['AIThreat'].value_counts().dropna()

ai_threat_df = pd.DataFrame({
    'Response': ai_threat.index,
    'Count': ai_threat.values
})

threat_mapping = {
    "I'm not sure": 'Uncertain',
    'Yes': 'Concerned',
    'No': 'Not Concerned'
}

ai_threat_df['Response'] = ai_threat_df['Response'].map(lambda x: threat_mapping.get(x, x))

custom_colors = ['#17B897', '#1DA886', '#229975']

fig3 = px.pie(
    ai_threat_df,
    names='Response',
    values='Count',
    color_discrete_sequence=custom_colors,
    title='AI Threat Perception Among Data Professionals in 2025<br><sub>Most professionals are not concerned about AI impact on jobs</sub>'
)

fig3.update_traces(
    texttemplate='%{label}<br>%{percent:.0%}',
    textposition='inside',
    textfont=dict(size=17, color='white'),
    marker=dict(line=dict(color='white', width=3)),
    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
)

fig3.update_layout(
    template='plotly_white',
    height=800,
    width=800,
    title_font=dict(size=26),
    showlegend=False,
    margin=dict(l=20, r=20, t=110, b=40)
)

fig3.show()

---
### Programming languages

In [ ]:
# Configuration
TOP_N = 10
MIN_SALARY = 10_000
MAX_SALARY = 500_000

# Get top languages by usage
languages = df_ds['LanguageHaveWorkedWith'].dropna().str.split(';').explode()
top_langs = languages.value_counts().head(TOP_N).index

# Calculate usage percentage
usage_pct = (languages.value_counts()[top_langs] / len(df_ds) * 100).round(1)

# Calculate median salary for each language
df_expanded = (
    df_ds[df_ds['LanguageHaveWorkedWith'].notna()]
    .assign(Language=lambda x: x['LanguageHaveWorkedWith'].str.split(';'))
    .explode('Language')
)

salaries = (
    df_expanded[
        (df_expanded['Language'].isin(top_langs)) &
        (df_expanded['ConvertedCompYearly'].between(MIN_SALARY, MAX_SALARY))
    ]
    .groupby('Language')['ConvertedCompYearly']
    .median()
)

# Combine data
data = pd.DataFrame({
    'Language': usage_pct.index,
    'Usage_Pct': usage_pct.values,
    'Salary': salaries.reindex(usage_pct.index).values
}).sort_values('Usage_Pct', ascending=True)

# Create visualization
fig = px.bar(
    data,
    x='Usage_Pct',
    y='Language',
    orientation='h',
    color='Salary',
    color_continuous_scale='blugrn',
    text=data['Usage_Pct'].apply(lambda x: f'{int(x)}%'),
    labels={'Usage_Pct': 'Usage Percentage', 'Salary': 'Median Salary'},
    title='Programming Languages: Usage & Compensation in 2025<br><sub>Bar length shows usage %, color indicates median salary</sub>'
)

fig.update_traces(
    textposition='outside',
    textfont_size=16,
    hovertemplate='<b>%{y}</b><br>Usage: %{x:.1f}%<br>Median Salary: $%{customdata[0]:,.0f}<extra></extra>',
    customdata=data[['Salary']].values
)

fig.update_layout(
    template='plotly_white',
    height=550,
    font=dict(size=16),
    title_font=dict(size=26),
    margin=dict(l=20, r=150, t=110, b=60),
    xaxis=dict(visible=False),
    yaxis_title='',
    coloraxis_colorbar=dict(
        title='Salary<br>(USD)',
        tickprefix='$',
        tickformat=',.0f',
        title_font=dict(size=16),
        tickfont=dict(size=14)
    )
)

fig.show()

---
### Remote working

In [ ]:
# Prepare data for salary analysis by experience and remote work
df_combined = df_ds[
    (df_ds['ConvertedCompYearly'].notna()) & 
    (df_ds['ConvertedCompYearly'] > 0) &
    (df_ds['ConvertedCompYearly'] < 500000) &
    (df_ds['WorkExp'].notna()) &
    (df_ds['RemoteWork'].notna())
].copy()

# Create experience groups
def categorize_experience(years):
    try:
        years = float(years)
        if years < 3:
            return '0-2 years'
        elif years < 6:
            return '3-5 years'
        elif years < 11:
            return '6-10 years'
        else:
            return '11+ years'
    except:
        return None

# Categorize remote work into three groups
def categorize_remote_work(status):
    if 'Remote' in str(status):
        return 'Remote'
    elif 'Hybrid' in str(status):
        return 'Hybrid'
    else:
        return 'On-site'

df_combined['ExpGroup'] = df_combined['WorkExp'].apply(categorize_experience)
df_combined['RemoteWorkGroup'] = df_combined['RemoteWork'].apply(categorize_remote_work)
df_combined = df_combined[df_combined['ExpGroup'].notna()]

# Calculate median salary by experience and remote work
salary_exp_remote = df_combined.groupby(['ExpGroup', 'RemoteWorkGroup'])['ConvertedCompYearly'].agg(['median', 'count']).reset_index()
salary_exp_remote.columns = ['ExpGroup', 'RemoteWork', 'Median_Salary', 'Count']

# Filter out small samples
salary_exp_remote = salary_exp_remote[salary_exp_remote['Count'] >= 10]

exp_order = ['0-2 years', '3-5 years', '6-10 years', '11+ years']

In [ ]:
# Heatmap: Salary by Experience and Remote Work
# Pivot the data for heatmap
salary_pivot = salary_exp_remote.pivot(index='RemoteWork', columns='ExpGroup', values='Median_Salary')
salary_pivot = salary_pivot[exp_order]  # Reorder columns

# Remove column name and reorder rows
salary_pivot.columns.name = None
row_order = ['Remote', 'Hybrid', 'On-site']
salary_pivot = salary_pivot.reindex([row for row in row_order if row in salary_pivot.index])

fig_heatmap = px.imshow(
    salary_pivot,
    labels=dict(x="", y="", color="Median Salary"),
    x=exp_order,
    y=salary_pivot.index,
    color_continuous_scale='blugrn',
    aspect='auto',
    title="Salary by Experience and Work Arrangement in 2025<br><sub>Median Compensation by Years of Experience and Remote Work Status</sub>"
)

fig_heatmap.update_traces(
    text=salary_pivot.values,
    texttemplate='$%{text:.0f}',
    textfont_size=16,
    hovertemplate='<b>%{y}</b><br>%{x}<br>Median Salary: $%{z:,.0f}<extra></extra>'
)

fig_heatmap.update_layout(
    template='plotly_white',
    height=550,
    font=dict(size=16),
    title_font=dict(size=26),
    xaxis=dict(tickfont=dict(size=16)),
    yaxis=dict(tickfont=dict(size=16)),
    coloraxis_colorbar=dict(title="Salary<br>(USD)", tickformat='$,.0f', title_font=dict(size=16), tickfont=dict(size=15)),
    margin=dict(l=20, r=20, t=110, b=60)
)

fig_heatmap.show()

---
### Industries

In [ ]:
# Get top industries by count (excluding "Other")
industry_counts = df_ds[df_ds['Industry'] != 'Other:']['Industry'].value_counts().head(10)

# Calculate percentage
industry_pct = (industry_counts / len(df_ds) * 100).round(1)

# Calculate median salary for each industry
df_industry_filtered = df_ds[
    (df_ds['Industry'].isin(industry_pct.index)) &
    (df_ds['Industry'] != 'Other:') &
    (df_ds['ConvertedCompYearly'].between(10000, 500000))
]

industry_salaries = (
    df_industry_filtered
    .groupby('Industry')
    .agg({'ConvertedCompYearly': ['median', 'count']})
)
industry_salaries.columns = ['Median_Salary', 'Count']
industry_salaries = industry_salaries[industry_salaries['Count'] >= 20]

# Combine data
data = pd.DataFrame({
    'Industry': industry_pct.index,
    'Usage_Pct': industry_pct.values,
    'Salary': industry_salaries.reindex(industry_pct.index)['Median_Salary'].values
}).sort_values('Usage_Pct', ascending=True)

# Create visualization
fig = px.bar(
    data,
    x='Usage_Pct',
    y='Industry',
    orientation='h',
    color='Salary',
    color_continuous_scale='blugrn',
    text=data['Usage_Pct'].apply(lambda x: f'{int(x)}%'),
    labels={'Usage_Pct': 'Percentage of Professionals', 'Salary': 'Median Salary'},
    title='Industries: Employment & Compensation in 2025<br><sub>Bar length shows % of data professionals, color indicates median salary</sub>'
)

fig.update_traces(
    textposition='outside',
    textfont_size=16,
    hovertemplate='<b>%{y}</b><br>Employment: %{x:.1f}%<br>Median Salary: $%{customdata[0]:,.0f}<extra></extra>',
    customdata=data[['Salary']].values
)

fig.update_layout(
    width=1200,
    height=627,
    template='plotly_white',
    font=dict(size=16),
    title_font=dict(size=26),
    margin=dict(l=20, r=150, t=110, b=60),
    xaxis=dict(visible=False),
    yaxis_title='',
    coloraxis_colorbar=dict(
        title='Salary<br>(USD)',
        tickprefix='$',
        tickformat=',.0f',
        title_font=dict(size=16),
        tickfont=dict(size=14)
    )
)


fig.show()

# **2. Salary Trends: 2020 - 2025**

In [ ]:
# Getting data about all years
folder = "Stack-overflow-datasets"

dfs = []
for f in glob.glob(f"{folder}/*"):
    year = re.search(r"(20\d{2})", os.path.basename(f))
    year = int(year.group(1)) if year else None

    df = pd.read_csv(f, low_memory=False)
    df["Year"] = year
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
print(df_all.shape)
df_all.head(1)

In [ ]:
# Mark rows where DevType contains any of the data roles, here are more than in 2025
data_roles = [
    'Data scientist or machine learning specialist',
    'Engineer, data',
    'Data or business analyst',
    "Data engineer",
    "Data scientist",
    "AI/ML engineer",
    "Data or business analyst",
    "Financial analyst or engineer"
]

df_all['IsDataJob'] = df_all['DevType'].astype(str).apply(
    lambda x: any(k in x for k in data_roles))

In [ ]:
# These roles are chosen to broadly cover data professionals
# Filter only data workers
df_all_ds = df_all[df_all['IsDataJob']]

In [ ]:
# Data cleaning for salary trend
df_clean = df_all_ds[['Year', 'ConvertedCompYearly']].dropna()
df_clean['Year'] = df_clean['Year'].astype(int)
df_clean['ConvertedCompYearly'] = df_clean['ConvertedCompYearly'].astype(float)

yearly = (
    df_clean
    .groupby('Year')['ConvertedCompYearly']
    .median()
    .reset_index()
)

In [ ]:
# Create line chart with Plotly
fig_trend = px.line(
    yearly,
    x='Year',
    y='ConvertedCompYearly',
    markers=True,
    title='Data Professional Salaries: Median Growth in USD $'
)

fig_trend.update_traces(
    line=dict(color='#17B897', width=4),
    marker=dict(size=14, color='#17B897'),
    text=yearly['ConvertedCompYearly'].apply(lambda x: f'${int(x/1000)}k'),
    textposition='top center',
    mode='lines+markers+text',
    textfont=dict(size=18),
    hovertemplate='<b>Year: %{x}</b><br>Median Salary: $%{y:,.0f}<extra></extra>'
)

fig_trend.update_layout(
    template='plotly_white',
    height=550,
    font=dict(size=16),
    title_font=dict(size=26),
    yaxis=dict(range=[yearly['ConvertedCompYearly'].min() * 0.88, yearly['ConvertedCompYearly'].max() * 1.05], visible=False),
    xaxis=dict(dtick=1, showgrid=False, title='', tickfont=dict(size=18)),
    margin=dict(l=30, r=30, t=160, b=60)
)

fig_trend.show()